In [1]:
import os
from torch.utils.data import Dataset
import torch
#from wordPieceTokenizer import WordPieceTokenizer
import json
import re
import urllib.request
from collections import defaultdict, Counter
import random
import numpy as np
from sklearn.metrics import accuracy_score

random.seed(0)
import pandas as pd
def load_separate_and_clean_stories(filename):
    with open(filename, 'r') as file:
        content = file.read()

    stories = content.split('\n\n\n\n')

    cleaned_stories = []
    for story in stories:
        cleaned_story = re.sub(r'\n\s*\n', '\n', story.strip())
        cleaned_stories.append(cleaned_story)
    
    return cleaned_stories

def separate_sentences(text):
    text = text.replace('...','#^')
    text = text.replace('.','~.')
    text = text.replace('?','@?')
    text = text.replace('!','%!')
    
    b = re.split('[.?!^]' , text)                                                                                                                                                                                                                                                                                                                                          
    c = [w.replace('~', '.') for w in b]
    c = [w.replace('@', '?') for w in c]
    c = [w.replace('#', '...') for w in c]
    c = [w.replace('%', '!') for w in c]
    
    return(c)

/opt/miniconda3/envs/fAIrytale/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


### TOKENIZER

In [32]:
class WordPieceTokenizer():
    def __init__(self, vocab_size=10000):
        self.vocab = {}
        self.word_freqs = {}
        self.vocab_size = vocab_size
        self.unk_token = "[UNK]"
        self.aps_token = "[APS]"
        self.space_token = "[SPACE]"
        self.brk_token = "[BRK]"
        self.sep_token = "[SEP]"
        self.cls_token = "[CLS]"
        self.pad_token = "[PAD]"
        self.mask_token = "[MASK]"
        self.wordpieces_prefix ="##"
        self.word2idx = {}
        self.idx2word = {}

   
    def fit(self, text):
        # Count word frequencies
        text = re.sub(r'\n+', ' ' + self.brk_token + ' ', text)
        text = re.sub(r"\s'\s", self.space_token + self.aps_token + self.space_token, text)
        text = re.sub(r"\s'", self.space_token + self.aps_token, text)
        text = re.sub(r"'\s",  self.aps_token + self.space_token, text)
        # Change charcater ' to [APS]
        text = re.sub(r'\'', self.aps_token, text)
        words = re.findall(r'\w+[\w.,;!?\'\"-]*|[\.,;!?\'\"-]+', text)
        
        self.word_freqs = Counter(words)

        alphabet = []
        for word in self.word_freqs.keys():
            if word == self.brk_token or word == self.aps_token or word == self.space_token:
                continue 
            # Add the first letter of the word to the alphabet if not exists
            if word[0] not in alphabet:
                alphabet.append(word[0])
            # Add the rest of the letters to the alphabet if not exist with a prefix
            for letter in word[1:]:
                if f"##{letter}" not in alphabet:
                    alphabet.append(f"##{letter}")

        alphabet.sort()
        
        # Add special tokens to the vocabulary plus the created alphabet
        self.vocab = [self.unk_token, self.cls_token, self.sep_token, self.space_token, self.pad_token, self.mask_token, self.brk_token, self.aps_token ] + alphabet.copy()
        # Create a dictionary with all words and all splitted characters
        splits = {
            word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
            for word in self.word_freqs.keys()
        }
        
        while len(self.vocab) < self.vocab_size:
            scores = self._compute_pair_scores(splits)
            if not scores:
                break
            best_pair, max_score = "", None
            for pair, score in scores.items():
                if max_score is None or max_score < score:
                    best_pair = pair
                    max_score = score
            
            splits = self._merge_pair(*best_pair, splits)
            new_token = (
                best_pair[0] + best_pair[1][2:]
                if best_pair[1].startswith("##")
                else best_pair[0] + best_pair[1]
            )
            self.vocab.append(new_token)
        self.word2idx = {word: idx for idx, word in enumerate(self.vocab)}
        self.idx2word = {idx: word for idx, word in enumerate(self.vocab)}
        print(self.vocab)
        print(len(self.vocab))
        print(self.word_freqs)
    
    
    def encode(self, text):
        # Normalize and split the text
        text = re.sub(r'\n+', ' ' + self.brk_token + ' ', text)
        text = re.sub(r"\s'\s", self.space_token + self.aps_token + self.space_token, text)
        text = re.sub(r"\s'", self.space_token + self.aps_token, text)
        text = re.sub(r"'\s",  self.aps_token + self.space_token, text)
        # Change charcater ' to [APS] and
        text = re.sub(r'\'', self.aps_token, text)
        pattern = r'\w+[\w.,;!?\'\"-]*|[\.,;!?\'\"-]+|(?:' + re.escape(self.brk_token) + r'|' + re.escape(self.aps_token) + r'|' + re.escape(self.space_token) + r')'
        words = re.findall(pattern, text)
        
        # Tokenize into words and subwords
        tokens = []
        for word in words:
            if word in self.vocab:
                tokens.append(word)
            else:
                sub_tokens = self.tokenize_word(word)
                tokens.extend(sub_tokens)
    
        
        # Convert tokens to ids
        token_ids = []
        token_ids.extend(self.word2idx[token] for token in tokens if token in self.word2idx)

        return token_ids

    def tokenize_word(self, word):
        if word == self.brk_token:
            return [self.brk_token]
        if word == self.aps_token:
            return [self.aps_token]
        if word == self.space_token:
            return [self.space_token]
        
        subwords = []
        start = 0
        while start < len(word):
            match = False
            for end in range(len(word), start, -1):
                subword = word[start:end]
                if start > 0:
                    subword = "##" + subword
                if subword in self.vocab:
                    subwords.append(subword)
                    start = end
                    match = True
                    break
            if not match:  # No subword match found
                subwords.append(self.unk_token)
                break
        return subwords
    
    def add_special_tokens(self, token_ids1, token_ids2, max_length=60):
        tokens_with_special_tokens  = [self.word2idx[self.cls_token]] + token_ids1 + [self.word2idx[self.sep_token]] + token_ids2 + [self.word2idx[self.sep_token]]
        # Create attention mask
        attention_mask = [1] * len(tokens_with_special_tokens)

        # Create token segment type ids
        token_type_ids = [0] * (len(token_ids1) + 2) + [1] * (len(token_ids2) + 1)
        

        padded_token_ids = tokens_with_special_tokens + [self.word2idx[self.pad_token]] * (max_length - len(tokens_with_special_tokens))
        attention_mask = attention_mask + [0] * (max_length - len(attention_mask))
        token_type_ids = token_type_ids + [0] * (max_length - len(token_type_ids))
        
        return padded_token_ids, attention_mask, token_type_ids
    
    def decode(self, indices):
        tokens = [self.idx2word[index] for index in indices]
        # Split the text by the first sep_token
        sep_index = tokens.index(self.sep_token)
        sentence1 = tokens[1:sep_index]
        sentence2 = tokens[sep_index + 1:]
        text1 = ''
        text2 = ''
        # Perform for loop for both sentences at the same time
        

        for token in sentence1:
            if token.startswith(self.wordpieces_prefix):
                # Remove the '##' prefix and concatenate without space
                text1 += token[2:]
            elif token in [self.unk_token, self.cls_token, self.sep_token, self.pad_token, self.mask_token]:
                # Skip special tokens if desired, or handle them differently
                continue
            elif token == self.aps_token:
                # Replace [APS] with a ' character
                text1 += "'"
            elif token == self.space_token:
                # Replace [SPACE] with a space character
                text1 += ' '
            elif token == self.brk_token:
                # Replace [BRK] with a newline character
                text1 += '\n'
            else:
                # Add a space before the token if it's not the first token and the last character isn't a newline
                if text1 and not text1.endswith('\n') and not text1.endswith("'"):
                    text1 += ' '
                text1 += token

        for token in sentence1:
            if token.startswith(self.wordpieces_prefix):
                # Remove the '##' prefix and concatenate without space
                text2 += token[2:]
            elif token in [self.unk_token, self.cls_token, self.sep_token, self.pad_token, self.mask_token]:
                # Skip special tokens if desired, or handle them differently
                continue
            elif token == self.aps_token:
                # Replace [APS] with a ' character
                text2 += "'"
            elif token == self.space_token:
                # Replace [SPACE] with a space character
                text2 += ' '
            elif token == self.brk_token:
                # Replace [BRK] with a newline character
                text2 += '\n'
            else:
                # Add a space before the token if it's not the first token and the last character isn't a newline
                if text2 and not text2.endswith('\n') and not text2.endswith("'"):
                    text2 += ' '
                text2 += token

        return text1, text2
    
    def save(self, path):
       with open(path, 'w') as f:
            json.dump({
                'vocab': self.vocab,
                'word_freqs': self.word_freqs,
                'word2idx': self.word2idx,
                'idx2word': self.idx2word,
            }, f, ensure_ascii=False)

    def load(self, path):
        with open(path, 'r') as f:
            data = json.load(f)
            self.vocab =  data['vocab']
            self.vocab_size = len(self.vocab)
            self.word_freqs =  {k: int(v) for k, v in data['word_freqs'].items()}
            self.word2idx =  {k: int(v) for k, v in data['word2idx'].items()}
            self.idx2word = {int(k): v for k, v in data['idx2word'].items()}
            
        
    def _compute_pair_scores(self, splits):
        letter_freqs = defaultdict(int)
        pair_freqs = defaultdict(int)
        # Compute the frequency of each letter and pair of consecutive letters
        for word, freq in self.word_freqs.items():
            split = splits[word]
            if len(split) == 1:
                letter_freqs[split[0]] += freq
                continue
            for i in range(len(split) - 1):
                pair = (split[i], split[i + 1])
                letter_freqs[split[i]] += freq
                pair_freqs[pair] += freq
            letter_freqs[split[-1]] += freq

        # Compute the score of each pair (pair frequency / (letter1 frequency * letter2 frequency)
        scores = {
            pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
            for pair, freq in pair_freqs.items()
        }
        return scores
    
    def _merge_pair(self, a, b, splits):
        for word in self.word_freqs:
            split = splits[word]
            if len(split) == 1:
                continue
            i = 0
            while i < len(split) - 1:
                if split[i] == a and split[i + 1] == b:
                    merge = a + b[2:] if b.startswith("##") else a + b
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[word] = split
        return splits

def mask_tokens(token_ids, tokenizer):
    gt = token_ids.copy()
    # Mask 15% of the tokens
    masked_indices = set()
    # 15% of significant tokens, different to [CLS], [SEP], and [PAD]
    significant_tokens = [token for token in token_ids if token not in [tokenizer.word2idx[tokenizer.cls_token], tokenizer.word2idx[tokenizer.sep_token], tokenizer.word2idx[tokenizer.pad_token]]]
    num_masked = max(1, int(len(significant_tokens) * 0.15))
    while len(masked_indices) < num_masked:
        index = random.randint(1, len(token_ids) - 2)
        if index in masked_indices:
            continue
        token = token_ids[index]
        if token in [tokenizer.word2idx[tokenizer.cls_token], tokenizer.word2idx[tokenizer.sep_token], tokenizer.word2idx[tokenizer.pad_token]]:
            continue
        token_ids[index] = tokenizer.word2idx[tokenizer.mask_token]
        masked_indices.add(index)
    
    labels = [-100 if i not in masked_indices else gt[i] for i in range(len(token_ids))]
    return token_ids, labels


def load_separate_and_clean_stories(filename):
    with open(filename, 'r') as file:
        content = file.read()

    stories = content.split('\n\n\n\n')

    cleaned_stories = []
    for story in stories:
        cleaned_story = re.sub(r'\n\s*\n', '\n', story.strip())
        cleaned_stories.append(cleaned_story)
    
    return cleaned_stories

## V.1

In [21]:
class MyDataset(Dataset):

    def __init__(self, dict_,mask=False):
        super().__init__()
        self.dict = dict_
        self.mask = mask
    def __len__(self):
        return len(self.dict)

    def __getitem__(self, idx):
        
        text = self.dict[str(idx)]['text']

        tokens = torch.tensor(self.dict[str(idx)]['tokens'])

        if self.mask!=False:
            idxs = np.linspace(0,len(tokens)-1,len(tokens)).astype(int)
            pos = random.choices(idxs, k=int(len(idxs)*0.15))
            tokens[pos] = self.mask

        return text,tokens

In [4]:
# Load the tokenizer
tokenizer = WordPieceTokenizer()
tokenizer.load('wordPieceVocab.json')

# Load the dataset
dataset_txt = load_separate_and_clean_stories("dataset/combined_stories.txt")

dict_ = {}
for i in range(len(dataset_txt)//20):
    dict_[i] = {'text':dataset_txt[i],
                'tokens':tokenizer.encode_n(dataset_txt[i])
                 }

with open("dataset/dataset_dict.json", "w") as outfile: 
    json.dump(dict_, outfile)  # We can read this file to avoid computing again the dict_

In [22]:
with open('dataset/dataset_dict.json') as json_file:
    dict_ = json.load(json_file)
    
complete_dataloader = MyDataset(dict_,tokenizer.word2idx["[MASK]"])

In [24]:
len(complete_dataloader.__getitem__(0)[1])

1597

In [25]:
len(np.where(complete_dataloader.__getitem__(0)[1]==4)[0])

223

## V.2

In [43]:
class MyDataset(Dataset):

    def __init__(self, dataset,sentences):
        super().__init__()
        self.dataset = dataset
        self.sentences = sentences
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        
        title = self.dataset.iloc[idx]['Title']
        text = separate_sentences(self.dataset.iloc[idx]['cleaned_story'])
        list_sentences = [''.join(map(str, text[i:i+self.sentences])) for i in range(0, len(text), self.sentences)]

        return title,text,list_sentences

In [44]:
dataset = pd.read_csv('dataset/merged_stories(1).csv')
dataset.head(5)

,Unnamed: 0,Title,cleaned_story
0,0,Thumbelina,"Once upon a time, in a world of wonder and enc..."
1,1,The Star Money,"Once upon a time, in a quaint village nestled ..."
2,2,The Twelve Dancing Princesses,In a kingdom where castles touched the clouds ...
3,3,The Elves and the Shoemaker,In a quaint village nestled at the edge of a l...
4,4,The Fox and the Cat,"Once upon a time, in a lush forest filled with..."


In [47]:
complete_dataloader = MyDataset(dataset,6)
print(complete_dataloader.__getitem__(0)[0])
print(len(complete_dataloader.__getitem__(0)[1]))
print(len(complete_dataloader.__getitem__(0)[2]))


Thumbelina
60
10


In [58]:
t = 'My text...My text. My text! My text?'
separate_sentences(t)

['My text...', 'My text.', ' My text!', ' My text?', '']

## V.3

In [2]:
class Custom_Dataset(Dataset):

    def __init__(self, dataset,sentences):
        super().__init__()
        self.dataset = dataset
        self.sentences = sentences
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        
        title = self.dataset.iloc[idx]['Title']
        text = separate_sentences(self.dataset.iloc[idx]['cleaned_story'])
        list_sentences = [''.join(map(str, text[i:i+self.sentences])) for i in range(0, len(text), self.sentences)]
        it = random.randint(0,len(list_sentences)-2)
        sentence = list_sentences[it]

        if random.random()<0.5:
            next_sentence = list_sentences[it+1]
            is_next = True
            
        else:
            idx2 = idx
            while idx2 == idx:
                idx2 = random.randint(0,len(self.dataset)-1)
            text2 = separate_sentences(self.dataset.iloc[idx2]['cleaned_story'])
            list_sentences2 = [''.join(map(str, text2[i:i+self.sentences])) for i in range(0, len(text2), self.sentences)]
            it = random.randint(0,len(list_sentences2)-1)
            next_sentence = list_sentences2[it]
            
            is_next = False
        return title,sentence,next_sentence,is_next

In [3]:
dataset = pd.read_csv('/Users/francesccarandellverdaguer/fairyTaleAI/dataset/merged_stories_full.csv')
dataset.head(5)

,Unnamed: 0,Title,cleaned_story
0,0,Thumbelina,"Once upon a time, in a world of wonder and enc..."
1,1,The Star Money,"Once upon a time, in a quaint village nestled ..."
2,2,The Twelve Dancing Princesses,In a kingdom where castles touched the clouds ...
3,3,The Elves and the Shoemaker,In a quaint village nestled at the edge of a l...
4,4,The Fox and the Cat,"Once upon a time, in a lush forest filled with..."


In [4]:
complete_dataloader = Custom_Dataset(dataset,2)
for i in range(len(dataset)):
    try:
        complete_dataloader.__getitem__(i)
    except Exception as e:
        print(e)
        print(dataset.iloc[i].Title)

empty range for randrange() (0, 0, 0)
Icarus and the Wax Wings: A Greek Myth


## DATASET ANALYSIS

In [10]:
dataset = pd.read_csv('/Users/francesccarandellverdaguer/fairyTaleAI/dataset/merged_stories_full.csv')
dataset.head(5)
tokenizer = WordPieceTokenizer()
tokenizer.load('/Users/francesccarandellverdaguer/fairyTaleAI/tokenizer/wordPieceVocab.json')

for idx in range(len(dataset)):
    text = separate_sentences(dataset.iloc[idx]['cleaned_story'])
    list_sentences = [''.join(map(str, text[i:i+2])) for i in range(0, len(text), 2)]
    for i in range(len(list_sentences)):
        enc = tokenizer.encode(list_sentences[i])

        if len(enc)>512:
            print(dataset.iloc[idx]['Title'],str(i))
       # if len(enc):
        #    print(dataset.iloc[idx]['Title'],str(i),str(len(enc)))
    if len(list_sentences)<=2:
        print(dataset.iloc[idx]['Title'],str(len(list_sentences)))


Don't Let the Pigeon Drive the Bus! 2


KeyboardInterrupt: 

In [9]:
dataset = pd.read_csv('/Users/francesccarandellverdaguer/fairyTaleAI/dataset/merged_stories_full.csv')
dataset.head(5)
tokenizer = WordPieceTokenizer()
tokenizer.load('/Users/francesccarandellverdaguer/fairyTaleAI/tokenizer/wordPieceVocab.json')

sub = dataset[dataset.Title=="The Barber's Tale of his First Brother."]
text = separate_sentences(sub.iloc[0]['cleaned_story'])
list_sentences = [''.join(map(str, text[i:i+2])) for i in range(0, len(text), 2)]
tokens = []
for s in list_sentences:
    tokens.append(tokenizer.encode(s))

In [8]:
list_sentences

["Know then, O Commander of the Faithful, that my first brother, Al Bakbuk, the Prattler, is a Hunchback who took to tailoring in Baghdad, and he used to sew in a shop hired from a man of much wealth, who dwelt over the shop, and there was also a flour-mill in the basement. One day as my brother, the Hunchback, was sitting in his shop a tailoring, he chanced to raise his head and saw a lady like the rising full moon at a balconied window of his landlord's house, engaged in looking out at the passers by.",
 ' When my brother beheld her, his heart was taken with love of her and he passed his whole day gazing at her and neglected his tailoring till eventide. Next morning he opened his shop and sat him down to sew; but, as often as he stitched a stitch, he looked to the window and saw her as before; and his passion and infatuation for her increased.',
 ' On the third day as he was sitting in his usual place gazing on her, she caught sight of him and, perceiving that he had been captivated 

In [12]:
for i in tokens:
    print(i)
    print('\n')

[110, 67, 68, 76, 5645, 58, 67, 12, 114, 102, 68, 66, 66, 54, 67, 57, 58, 71, 2967, 5645, 58, 105, 54, 62, 7667, 59, 74, 65, 12, 5645, 54, 73, 139, 78, 132, 62, 71, 72, 73, 128, 71, 68, 7667, 58, 71, 12, 100, 65, 101, 54, 64, 55, 74, 64, 12, 5645, 58, 115, 71, 54, 73, 73, 65, 58, 71, 12, 135, 72, 127, 107, 74, 67, 6203, 55, 54, 3305, 7250, 68, 6849, 68, 64, 6849, 146, 54, 62, 65, 68, 71, 62, 9994, 135, 67, 8236, 127, 67, 57, 134, 58, 147, 72, 58, 57, 6849, 145, 58, 76, 135, 67, 127, 7632, 68, 69, 134, 62, 71, 58, 57, 132, 71, 68, 66, 127, 139, 54, 67, 2967, 139, 74, 6203, 149, 58, 54, 65, 7667, 12, 7250, 68, 130, 76, 58, 65, 73, 141, 75, 58, 71, 5645, 58, 7632, 68, 69, 12, 127, 67, 57, 5645, 58, 71, 58, 149, 54, 72, 127, 65, 72, 68, 127, 132, 65, 68, 74, 71, 13, 66, 62, 65, 65, 135, 67, 5645, 58, 128, 54, 72, 58, 66, 58, 67, 73, 14, 114, 67, 58, 130, 54, 78, 127, 72, 139, 78, 128, 71, 68, 7667, 58, 71, 12, 5645, 58, 107, 74, 67, 6203, 55, 54, 3305, 12, 149, 54, 72, 145, 62, 73, 73, 62,

In [15]:
len(tokenizer.encode(' Shouts of joy broke forth from the attendants when he was seen approaching, and some of them ran to tell the queen that the monkey was there, and that before long she would be as well as ever she was. In fact, so great was their relief that they gave the monkey such a kind welcome, and were so anxious to make him happy and comfortable, that he soon forgot all the fears that had beset him as to his fate, and was generally quite at his ease, though every now and then a fit of home-sickness would come over him, and he would hide himself in some dark corner till it had passed away.'))


422

In [18]:
len(tokenizer.encode('It was during one of these attacks of sadness that a jelly-fish happened to swim by. At that time jelly-fishes had shells.'))
    

87

In [19]:
422+87

509

In [20]:
len(tokenizer.encode('In the evening she returned to the farm more angry than ever, and quite determined to revenge herself on the brownie whenever she had the chance, which happened to be very soon. It was the spring, and just the time of year when the dwarfs held their fete, so one day the brownie asked Jegu if he might bring his friends to have supper in the great barn, and whether he would allow them to dance there.'))
                     

278

In [21]:
len(tokenizer.encode('  Smearing his face with dirt, and tearing his clothes so as to look like a beggar, he stopped the passers by and, on pretence of seeking money or food, he questioned them about the village gossip.  In this manner he learned that his brothers had become great men, much respected in all the country round.'))

231

In [22]:
278 + 231

509

In [26]:
for t in dataset.Title.unique():

    sub = dataset[dataset.Title==t]
    text = separate_sentences(sub.iloc[0]['cleaned_story'])
    list_sentences = [''.join(map(str, text[i:i+2])) for i in range(0, len(text), 2)]
    tokens = []
    for s in list_sentences:
        tokens.append(tokenizer.encode(s))

    for tok in range(len(tokens)-1):
        if len(tokens[tok]) + len(tokens[tok+1]) >= 509:
            print(t,str(tok),str(len(tokens[tok]) + len(tokens[tok+1])))



The Elves and the Shoemaker 23 516
The Runaway Bunny 11 599
Midnight’s Canvas: The Path to Dawn 4 552
The Love Story of The Shepherdess and The Chimney Sweep 2 576
The Love Story of The Shepherdess and The Chimney Sweep 3 513
Medusa Before the Curse 0 560
The Story of Achilles 0 561
The Iron Stove a Fairy Tale about Love 0 542
The Iron Stove a Fairy Tale about Love 1 530
The Man Who Planted Trees 9 529
The Man Who Planted Trees 10 683
The little Peasant Fairy Tale 25 511
The Price of Time: A Tale of Love and Wealth 0 534
The frog king or Iron Henry 0 666
The frog king or Iron Henry 5 545
The frog king or Iron Henry 23 509
The frog king or Iron Henry 24 578
The frog king or Iron Henry 25 523
Our Lady's Child 22 517
Our Lady's Child 27 539
Our Lady's Child 28 564
Our Lady's Child 30 566
Our Lady's Child 36 556
The story of the youth who went forth to learn what fear was 60 542
Trusty John 4 630
Trusty John 18 529
Trusty John 43 602
Trusty John 44 514
Trusty John 49 511
Trusty John 58 537

KeyboardInterrupt: 

# MLM ACCURACY

In [33]:
class Custom_Dataset(Dataset):

    def __init__(self, dataset, sentences, tokenizer):
        super().__init__()
        self.dataset = dataset
        self.sentences = sentences
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        
        while True:
            title = self.dataset.iloc[idx]['Title']
            text = separate_sentences(self.dataset.iloc[idx]['cleaned_story'])
            list_sentences = [''.join(map(str, text[i:i+self.sentences])) for i in range(0, len(text), self.sentences)]
            
            
            if len(list_sentences[-1]) == 0:
                list_sentences.pop()

            if len(list_sentences) < 2:
                idx = random.randint(0, len(self.dataset) - 1)
                continue

            it = random.randint(0, len(list_sentences) - 2)

            sentence = list_sentences[it]

            if random.random() < 0.3:
                next_sentence = list_sentences[it + 1]
                is_next = 1
            else:
                idx2 = idx
                while idx2 == idx:
                    idx2 = random.randint(0, len(self.dataset) - 1)

                text2 = separate_sentences(self.dataset.iloc[idx2]['cleaned_story'])
                list_sentences2 = [''.join(map(str, text2[i:i + self.sentences])) for i in range(0, len(text2), self.sentences)]
                
                if len(list_sentences2) == 0:
                    continue
                
                if len(list_sentences2[-1]) == 0:
                    list_sentences2.pop()
                
                it = random.randint(0, len(list_sentences2) - 1)
                next_sentence = list_sentences2[it]
                is_next = 0            

            token_ids_sentence1 = self.tokenizer.encode(sentence)
            token_ids_sentence2 = self.tokenizer.encode(next_sentence)
            input_ids, attention_mask, segment_ids = self.tokenizer.add_special_tokens(token_ids_sentence1, token_ids_sentence2, max_length=512)
            masked_input_ids, labels = mask_tokens(input_ids, self.tokenizer)

            if len(masked_input_ids) == 512 and (len(token_ids_sentence1) + len(token_ids_sentence2)) < 509:
                break

        return title, torch.tensor(masked_input_ids), torch.tensor(attention_mask), torch.tensor(segment_ids), torch.tensor([is_next]), torch.tensor(labels)

In [34]:
dataset = pd.read_csv('/Users/francesccarandellverdaguer/fairyTaleAI/dataset/merged_stories_full.csv')
dataset.head(5)
tokenizer = WordPieceTokenizer()
tokenizer.load('/Users/francesccarandellverdaguer/fairyTaleAI/tokenizer/wordPieceVocab.json')

#sub = dataset[dataset.Title=="The Barber's Tale of his First Brother."]
#text = separate_sentences(sub.iloc[0]['cleaned_story'])
#list_sentences = [''.join(map(str, text[i:i+2])) for i in range(0, len(text), 2)]
#tokens = []
#for s in list_sentences:
 #   tokens.append(tokenizer.encode(s))

In [35]:
train_dataset = Custom_Dataset(dataset, 2, tokenizer)



In [36]:
pieces = train_dataset.__getitem__(0)
b = torch.ones(np.shape(pieces[-1]))

In [37]:
pieces[-1]

tensor([-100, -100,   58, -100,   67, -100,   76, -100, -100, -100, -100, -100,
        -100,   57, -100,    7, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100,   58, -100, -100, -100, -100,  130, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100,   58, -100, -100, -100, -100,   65, -100,
        -100, -100, -100, -100,   58, -100, -100,   12, -100, -100, -100, -100,
        -100,   75,   62, -100, -100, -100, -100, -100, -100,   54, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100,  132, -100, -100,   60,
         137, -100, -100, -100, -100, -100, -100, -100, -100,   67, -100,   58,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,   58,
         132, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, 3006, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -1

In [20]:
mask = pieces[-1] != -100 # these are the positions with a mask token 
filtered_labels = pieces[-1][mask]
filtered_predictions = b[mask]

In [23]:
filtered_predictions

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [25]:
pieces[-1]

tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,    5,
        -100, -100, -100, -100, -100,    5, -100,    5, -100, -100, -100, -100,
        -100,    5, -100, -100, -100,    5, -100,    5, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100,    5, -100, -100, -100, -100, -100,
        -100, -100, -100, -100,    5, -100,    5, -100, -100,    5, -100, -100,
        -100, -100, -100,    5,    5, -100,    5, -100,    5, -100, -100, -100,
        -100,    5, -100, -100, -100, -100,    5, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100,    5, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -1

In [7]:
def accuracy_mlm(preds,labels):
    
    labels_flat = labels.view(-1)
    preds_flat = preds.view(-1, preds.size(-1)).argmax(dim=1)
    
    # Identify positions where labels are not 0 (masked positions)
    mask = labels_flat != 0
    
    # Filter labels and predictions at masked positions
    filtered_labels = labels_flat[mask]
    filtered_predictions = preds_flat[mask]
    
    # Calculate accuracy
    accuracy = accuracy_score(filtered_labels.cpu().numpy(), filtered_predictions.cpu().numpy())
    
    return accuracy


preds = torch.tensor([[1,1,1,1],[0,0,0,0],[2,2,2,2],[0,1,0,1]])
labels = torch.tensor([[0,1,0,1],[0,1,0,1],[0,1,0,1],[0,1,0,1]])

In [13]:
labels_flat = labels.view(-1)
preds_flat = preds.view(-1) #preds.view(-1, preds.size(-1)).argmax(dim=1)

# Identify positions where labels are not 0 (masked positions)
mask = labels_flat != 0

# Filter labels and predictions at masked positions
filtered_labels = labels_flat[mask]
filtered_predictions = preds_flat[mask]

# Calculate accuracy
accuracy = accuracy_score(filtered_labels.cpu().numpy(), filtered_predictions.cpu().numpy())

In [14]:
accuracy

0.5

## CSV ALL SENTENCES

In [ ]:
def separate_sentences(text):
    text = text.replace('...','#^')
    text = text.replace('.','~.')
    text = text.replace('?','@?')
    text = text.replace('!','%!')
    
    b = re.split('[.?!^]' , text)                                                                                                                                                                                                                                                                                                                                          
    c = [w.replace('~', '.') for w in b]
    c = [w.replace('@', '?') for w in c]
    c = [w.replace('#', '...') for w in c]
    c = [w.replace('%', '!') for w in c]
    
    return(c)

In [9]:
dataset = pd.read_csv('/Users/francesccarandellverdaguer/fairyTaleAI/dataset/merged_stories_full.csv')
dataset.head(5)

ts = []
sents = []

dataset_s = pd.DataFrame(columns=['Title','Sentence'])

for i in range(len(dataset)):
    
    sentences = separate_sentences(dataset.iloc[i].cleaned_story)

    for j in range(len(sentences)):
        ts.append(dataset.iloc[i].Title)
        sents.append(sentences[j])

dataset_s.Title = ts
dataset_s.Sentence = sents


In [12]:
dataset_s.to_csv('dataset_sentences.csv')

In [13]:
dataset_s

,Title,Sentence
0,Thumbelina,"Once upon a time, in a world of wonder and enc..."
1,Thumbelina,"She was no bigger than a thumb, with eyes as ..."
2,Thumbelina,"Thumbelina's story is filled with magic, adve..."
3,Thumbelina,"One day, a kindly old woman, who loved to pla..."
4,Thumbelina,"The gift was a shimmering, mystical seed."
...,...,...
129702,The Lad With The Goat-Skin,"Well, I suppose, before they were married, To..."
129703,The Lad With The Goat-Skin,Whether he ever lost his time learning them s...
129704,The Lad With The Goat-Skin,MAN OR WOMAN BOY OR GIRL THAT READS WHAT FOLL...
129705,The Lad With The Goat-Skin,BATTEN DREW THIS AUG.
